# **DataSheet Description**

---


En este proyecto emplearemos un dataset que contiene información de los resultados de las carreras de el panorama nacional de carreras de caballos.

- **Puesto** (Clase): Posición en la que terminan la carrera **int64**
- **NombreCaballo**: Nombre de caballo  **object**
- **Peso** : Peso de caballo  **object**
- **Edad**: Edad de caballo **int64**
- **DistanciaMeta**: Distancia respecto a ganador en llegada a meta **object** (DROPEAR)
- **Mantilla**: Dorsal   **int64**
- **Dividendo** : Pago por euro apostado por ganador de carrera **object** (DROPEAR)
- **Propietario** : Cuadra Propietaria **object**
- **Preparador**: Nombre Preparador **object**
- **Jinete**: Jinete o Amazona que monta al caballo en esta carrera  **object**
- **Problemas**: Problemas que posee en esta carrera el caballo  **object**
- **Fecha**: Fecha Carrera   **object**
- **Hora**: Hora carrera  **object**
- **Terreno**: Estado del terreno en carrera **object**
- **Distancia**: Longitud carrera  **float64**
- **Tipo**: Tipo de terreno en carrera  **object**
- **SentidoHipodromo**:  (0 -> Sentido Normal, 1 -> Inverso) **int64**
- **Meteorología**: Previsión estado meteorología del día entero **object** (Tiempo3)
- **Lluvia**: Previsión cantidad de lluvia del dia entero  **object**
- **Viento**: Previsión velocidad de viento del dia entero  **int64**
- **Temperatura**: Previsión de temp. del horario de la carrera  **int64**
- **Hipodromo**: Nombre del hipódromo de la carrera   **object**

In [ ]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.compose
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

url = 'https://raw.githubusercontent.com/SergioNietoP/machine-learning-datasheets/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Train%20-%2012.csv'

url1 = 'https://raw.githubusercontent.com/SergioNietoP/machine-learning-datasheets/main/Learning%20Datasheets/Turf%20Prediction/Data%20Turf%20Races%20Test%20-%205.csv'
test = pd.read_csv(url1, sep=',')

test_labels = test.pop('Puesto')


train = pd.read_csv(url, sep=',')

train = train.drop('DistanciaMeta', axis=1)
train = train.drop('Dividendo', axis=1)

ogData = train
train.head()
train.count()



# Preprocesamiento
---

In [ ]:
pd.set_option('display.max_columns', None)
train.head(10)


## **Tratamiento (Puesto)(Train)**

In [ ]:
train = train.drop(train[train.Puesto >= 90].index)
train = train.reset_index(drop=True)
train.count()
# train['Puesto'] = train['Puesto'].replace(95, 10) # Penalización Parado

## **Tratamiento (Fecha)**


In [ ]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':  
   return '01'
  elif month == 'FEBRERO':
    return '02'            
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04' 
  elif month == 'MAYO':
    return '05'                  
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1
   
def calculateSeason(month, index):
  # train['Invierno'] = 0
  # train['Primavera'] = 0
  # train['Verano'] = 0
  train['Otoño'] = 0
  # if(month >= 12 and month <=2):
  #    train.at[index, 'Invierno'] = month
  # if(month >= 3 and month <=5):
  #    train.at[index, 'Primavera'] = month
  # if(month >= 6 and month <=8):
  #    train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"] 
convertDate(t_array)
train['FechaAux'].head(120)

In [ ]:
train.head(5)

In [ ]:
import datetime

def daysSincePreviousRace(nombres):
  index = 0
  for caballo in nombres:
    indexNombre = 0
    nombresAparece = []
    for otroCaballo in nombres:
      if(otroCaballo == caballo):
        nombresAparece.append(indexNombre)
      indexNombre += 1

    day_actual = train.at[index, 'day']
    month_actual = train.at[index, 'month']
    year_actual = train.at[index, 'year']

    fechaCaballoActual = dt.datetime(int(year_actual), int(month_actual), int(day_actual))
    fechaMasReciente = dt.datetime(2020, 1, 1)
    indexAux2 = 0
    fechaCambiada = False
    for aparicion in nombresAparece:
      if(train.at[aparicion, 'FechaAux'] < fechaCaballoActual): 
        if(indexAux2 == 0):
          fechaMasReciente = train.at[aparicion, 'FechaAux']
          fechaCambiada = True
          indexAux2 += 1
        elif(train.at[aparicion, 'FechaAux'] > fechaMasReciente):
          fechaMasReciente =  train.at[aparicion, 'FechaAux']
    if(fechaCambiada):
      diferenciaDias = (fechaCaballoActual - fechaMasReciente).days
    else:
      diferenciaDias = 45 # Penalización si no aparece en carrera anterior
    train.at[index, 'DaysSincePreviousRace'] = diferenciaDias
    index += 1


eventos = train['NombreCaballo']
daysSincePreviousRace(eventos)
train['DaysSincePreviousRace'].head(35)
# Recorro los nombres de caballos y guardo en un array los index de las columnas ligadas a un nombre de Caballo y despues recorrer las fechas y cuando coincida el array comprobar si ese dato
# es mas antiguo que la fecha de la fila actual y en el caso de serlo compruebo si es la más antigua de entre las anteriores participaciones

## **Tratamiento (NombreCaballo)(Train)**

In [ ]:
def eraseBlank(list):
  aux = []
  for element in list:
    element = element.strip()
    aux.append(element)
  return aux

test['NombreCaballo'] = test['NombreCaballo'].str.partition('(')[0]
listaNombreCaballo = test['NombreCaballo'].unique().tolist()

listTestHorse= eraseBlank(listaNombreCaballo)

print(listTestHorse)

In [ ]:
listTestHorse.sort()
print(listTestHorse)

In [ ]:
test['Jinete'] = test['Jinete'].str.partition('(')[0]
lista = test['Jinete'].unique().tolist()

listTestJockey = eraseBlank(lista)

test['Preparador'] = test['Preparador'].str.partition('(')[0]
lista = test['Preparador'].unique().tolist()

listTestTrainer = eraseBlank(lista)

test['Propietario'] = test['Propietario'].str.partition('(')[0]
lista = test['Propietario'].unique().tolist()

listTestOwner = eraseBlank(lista)



In [ ]:
train.info()

In [ ]:
def changeNotFoundInListHorse(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      print("Encontrado:", element)
      train.at[index, 'NombreCaballo'] = 'OtherHorse'
    else:
      print("NOO Encontrado:", element)
      listaApariciones.append(element)
    index += 1
  return listaApariciones


train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
listTrainHorse = train['NombreCaballo'].tolist()
listTrainHorse= eraseBlank(listTrainHorse)

## Calculo Contricantes
---

In [12]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




KeyboardInterrupt: ignored

## Calculo Raiz Puesto 
---

In [ ]:
def calculoRaizPuesto(index):
  i = 0
  while(i < index):
    train.at[i, 'Puesto'] = float(((train.at[i, 'Puesto'] / (train.at[i, 'Contrincantes'])**(1. / 3.))))
    i += 1
  
train['Puesto'] = train['Puesto'].astype(float)
calculoRaizPuesto(len(train.index))


In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','Puesto','Contrincantes']].head(501)

## **Busqueda Caballos Ganadores en ciertas condiciones**
---

### Busqueda UltimasActuacionesAlter
---

In [ ]:
def ultimasActuacionesIter(indices):                      
  for i in range(indices):
    print(i)
    puesto = train.at[i, 'Puesto'] #Para Train
    print("·Puesto -><", puesto)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0 # Media carreras
    sum = 0.0 # Numero Carreras

    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      nombreAux= train.at[j, 'NombreCaballo']
      if((nombreAux == nombre and (fechaAux < fecha))):
        print("Valor Puesto-->", train.at[j, 'Puesto'])
        media += (train.at[j, 'Puesto'] )
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = puesto
      train.at[i, 'UltimasActuacionesAlter'] = media
    else:
      train.at[i, 'UltimasActuacionesAlter'] = media
    print(i, "-->", nombre, " MediaUltAct->", media) 

        

indices = len(train.index)
ultimasActuacionesIter(indices)

In [ ]:
train['UltimasActuacionesAlter'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Distancia
---

In [ ]:
def comprobacionTipoDistancia(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = train.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def destrezaDistancia(indices):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoDistancia(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoDistancia(j)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaDistancia'] = 1
    else:
      train.at[i, 'DestrezaDistancia'] = 0
        

indices = len(train.index)
trainOg = train
destrezaDistancia(indices)


#Len Intervalos
print(len(train[(train['Distancia'] < 1600).tolist()]))
print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))
print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
train['DestrezaDistancia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','DestrezaDistancia']].head(501)

### Busqueda Meteorología
---

In [ ]:
train['Meteorología'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoMeteorologia(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado  
  val = train.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "precipitaciones" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, listaMeteo):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoMeteorologia(i, listaMeteo)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoMeteorologia(j, listaMeteo)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaMeteo'] = 1
    else:
      train.at[i, 'DestrezaMeteo'] = 0
        

indices = len(train.index)
listaMeteo = train['Meteorología'].unique().tolist()
destrezaMeteorologia(indices, listaMeteo)

In [ ]:
train['DestrezaMeteo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Lluvia
---

In [ ]:
train['LLuvia'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoLLuvia(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1 
  val = train.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, listaLLuvia):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoLLuvia(i, listaLLuvia)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoLLuvia(j, listaLLuvia)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaLLuvia'] = 1
    else:
      train.at[i, 'DestrezaLLuvia'] = 0
        

indices = len(train.index)
listaLLuvia = train['LLuvia'].unique().tolist()
destrezaMeteorologia(indices, listaLLuvia)

In [ ]:
train['DestrezaLLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Temperatura
---

In [ ]:
train['Temperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTemperatura(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = train.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaTemperatura(indices, listaTemperatura):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTemperatura(i, listaTemperatura)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTemperatura(j, listaTemperatura)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTemperatura'] = 1
    else:
      train.at[i, 'DestrezaTemperatura'] = 0
        

indices = len(train.index)
listaTemperatura = train['Temperatura'].unique().tolist()
destrezaTemperatura(indices, listaTemperatura)

In [ ]:
train['DestrezaTemperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Viento
---

In [ ]:
train['Viento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoViento(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = train.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaViento(indices, listaViento):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoViento(i, listaViento)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoViento(j, listaViento)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaViento'] = 1
    else:
      train.at[i, 'DestrezaViento'] = 0
        

indices = len(train.index)
listaViento = train['Viento'].unique().tolist()
destrezaViento(indices, listaViento)

In [ ]:
train['DestrezaViento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Terreno
---

In [ ]:
train['Terreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTerreno(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = train.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    val = val.replace('H-Bueno', 'H - Bueno')
    train.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'Arena'):
      tipo = 0
   elif(val == 'Playa'):
      tipo = 3
   else:
      tipo = 4
  return tipo;

def destrezaTerreno(indices, listaTerreno):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTerreno(i, listaTerreno)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indices):
      fechaAux = train.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTerreno(j, listaTerreno)
      nombreAux= train.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (train.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (train.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTerreno'] = 1
    else:
      train.at[i, 'DestrezaTerreno'] = 0
        

indices = len(train.index)
listaTerreno = train['Terreno'].unique().tolist()
destrezaTerreno(indices, listaTerreno)

In [ ]:
train['DestrezaTerreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Peso)**

In [ ]:
train['Peso'] = train['Peso'].str.partition('-')[0]
train['Peso'] = train['Peso'].str[:2]



In [ ]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Edad)**

In [ ]:
train['Edad'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Mantilla)**

In [ ]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Propietario)(Train)**

In [ ]:
def changeNotFoundInListOwner(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Propietario'] = 'OtherOwner'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones
    
    

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

listTrainOwner = train['Propietario'].tolist()
listTrainOwner= eraseBlank(listTrainOwner)


## **Tratamiento (Preparador)(Train)**

In [ ]:
def changeNotFoundInListTrainer(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Preparador'] = 'OtherTrainer'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

listTrainTrainer = train['Preparador'].tolist()
listTrainTrainer= eraseBlank(listTrainTrainer)


## **Tratamiento (Jinete)(Train)**

In [ ]:
def changeNotFoundInListJockey(listTrain, listTest):
  index = 0
  listaApariciones = []
  for element in listTrain:
    if(element not in listTest):
      train.at[index, 'Jinete'] = 'OtherJockey'
    else:
      listaApariciones.append(element)
    index += 1
  return listaApariciones

#Lógica de lista de Test en "Tratamiento(NombreCaballo)"

train['Jinete'] = train['Jinete'].str.partition('(')[0]
listTrainJockey = train['Jinete'].tolist()
listTrainJockey= eraseBlank(listTrainJockey)


## **Tratamiento Change Not Found In Test(NombreCaballo)(Propieatario)(Preparador)(Jinete)**

In [ ]:
listAparicionesHorse= changeNotFoundInListHorse(listTrainHorse, listTestHorse)

train['Propietario'] = train['Propietario'].str.strip()
train['NombreCaballo'] = train['NombreCaballo'].str.strip()
train['Jinete'] = train['Jinete'].str.strip()
train['Preparador'] = train['Preparador'].str.strip()

train['NombreCaballo'].head(30)

In [ ]:
listAparicionesOwner = changeNotFoundInListOwner(listTrainOwner, listTestOwner)
train['Propietario'].head(15)

In [ ]:
listAparicionesTrainer = changeNotFoundInListTrainer(listTrainTrainer, listTestTrainer)
train['Preparador'].head(15)

In [ ]:
listAparicionesJockey = changeNotFoundInListJockey(listTrainJockey, listTestJockey)
train['Jinete'].head(15)

In [ ]:
train['NombreCaballo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Problemas)**


In [ ]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



## **Tratamiento (UltimasActuaciones)**


In [ ]:
# def count_lastRaces(arrayLast):
#   index = 0
#   for element in arrayLast:
#     element = element.replace(',', '')
#     element = element.replace('[', '')
#     element = element[:len(element)-1]
#     i = 0
#     j = 0
#     sum = 0
#     if 'Debutante' in element:
#       sum = 10
#       element = []
#     else:
#       element = element.replace('NP', '08')
#       element = element.replace('Desc', '08')
#       element = element.replace(' ', '')
#       while(i < len(element)):
#         if(i == 0 and len(element)>4):
#           j = j+2
#           sum += int(element[i:j])**1.3
#           i = i+2 
#         elif (i == 2 and len(element)>4):
#           j = j+2
#           sum += int(element[i:j])**1.2
#           i = i+2 
#         else:
#           j = j+2
#           print(index)
#           sum += int(element[i:j])
#           i = i+2
#       print('******INDEX ', index+2, '*****')
#       print('Sum -->', sum, '/Len -->', len(element))
#       sum = sum / len(element)
#       print('Result -->', sum, '/Element -->' ,element)
#     train.at[index, 'MediaUltimasActuaciones'] = sum
#     train.at[index, 'CantidadActuaciones'] = (len(element)/2)
#     index += 1

      



# t_array = train["UltimasActuaciones"]
# count_lastRaces(t_array)

In [ ]:
train[['NombreCaballo','UltimasActuacionesAlter']].head(128)

## **Tratamiento (Hora)**


In [ ]:
train['Hora'] = train['Hora'].str.replace(':', '') 

## **Tratamiento (Terreno)**


In [ ]:
train.info()

## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (Hipodromo)**


# Visualizacion

In [ ]:
train.describe()

In [ ]:
train.hist(figsize = (40,30))

In [ ]:
# train['Puesto'].corr(train['MediaUltimasActuaciones'])

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr
def kendall_pval(x,y):
    return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]


print("Correlation -->", train['Puesto'].corr(train['UltimasActuacionesAlter']))    
print(train['Puesto'].corr(train['UltimasActuacionesAlter'], method=kendall_pval))
print(train['Puesto'].corr(train['UltimasActuacionesAlter'], method=pearsonr_pval))   
print(train['Puesto'].corr(train['UltimasActuacionesAlter'], method=spearmanr_pval))

In [ ]:
C_mat = train.corr()
fig = plt.figure(figsize = (15,15))

#Correlation ranges from -1 to +1. Values closer to zero means there is no linear trend between the two variables. 
#The close to 1 the correlation is the more positively correlated they are; that is as one increases so does the other and the closer to 1 the stronger this relationship is. 
#A correlation closer to -1 is similar, but instead of both increasing one variable will decrease as the other increases.

sns.heatmap(C_mat, vmax = .8, square = True, cmap="PiYG")
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Distancia', y='Puesto')

plt.show()

In [ ]:
print(len(train[(train['Distancia'] < 1600).tolist()]))

print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))

print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Hora', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(50,8))
sns.boxplot(data=train, x='Tipo', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='Meteorología', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(40,8))
sns.boxplot(data=train, x='LLuvia', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Viento', y='Puesto')

plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=train, x='Temperatura', y='Puesto')

plt.show()

# Preprocesamiento Final

In [ ]:
train.info()

In [ ]:
pd.set_option('display.max_columns', None)
train.head(20)

In [ ]:
import sklearn
import sklearn.compose
#Solo ejecutar previo a esto product para separar las palabras
column_transformer_train = sklearn.compose.ColumnTransformer(transformers=[
    ("drop", "drop", ['Fecha','year','day', 'Problemas','Hora', 'FechaAux', 'Mantilla', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas','Hora']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'FechaAux', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()
X=train
X_TF = X #TensorFlow
X = X.drop(["Puesto"], axis = 1)

In [ ]:
X_transform = column_transformer_train.fit_transform(X)

In [ ]:
column_transformer_train.get_feature_names()

In [ ]:
train 
X_transform
X = X_transform       #Nos quedamos con todo menos la columna objetivo, en este caso 'price'
y = train["Puesto"]

In [ ]:
# train[['NombreCaballo','MediaUltimasActuaciones']].head(128)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1920)

# Optimización
---

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

#Regresión lineal
linear = LinearRegression()
linear.fit(X_train, y_train)

#Regresión de Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)

#LASSO

lasso = linear_model.Lasso()
lasso.fit(X_train, y_train)

#ElasticNet

elasticnet = ElasticNet()
elasticnet.fit(X_train, y_train);


reg = SGDRegressor(max_iter=50, random_state=43)
reg.fit(X_train, y_train);

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    elasticnet = ElasticNet(alpha=i*0.1, l1_ratio=j*0.1)
    elasticnet.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

for i in range(0, 11):
  for j in range(0, 11):
    #     xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") 148

    xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=i*0.1, reg_lambda=j*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact") #subsample ojo
    xgb_model.fit(X_train, y_train)
    print('ElasticNet (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
    print(f'{i} {j} ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', reg_alpha=7*0.1, reg_lambda=4*0.1, verbosity=0, random_state=42, eta=0.2, tree_method="exact")

xgb_model.fit(X, y)
#print('Lineal (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X))))
#print('Lineal (test): ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))

In [ ]:
!pip install catboost
import catboost as cb

In [ ]:
params = {"depth": 10, "learning_rate": 0.5, "iterations": 1, "l2_leaf_reg": 5} #Iterations CatBoost
model_cat_tun = cb.CatBoostClassifier(**params)
model_cat_tun.fit(X,y)

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor


# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 2000, random_state = 42)
# Train the model on training data
rf.fit(X, y);

In [ ]:

from sklearn.tree import DecisionTreeRegressor

# Build decision tree
tree=DecisionTreeRegressor(max_depth=20)
tree.fit(X, y);

In [ ]:
from sklearn.metrics import mean_absolute_error


print('RandomForest (train): ' + str(mean_absolute_error(y_train,  rf.predict(X_train))))
print('RandomForest (test): ' + str(mean_absolute_error(y_test,  rf.predict(X_test))))


print('---')


print('XGB (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
print('XGB (test): ' + str(mean_absolute_error(y_test, xgb_model.predict(X_test))))



print('---')

print('Cat (train): ' + str(mean_absolute_error(y_train, model_cat_tun.predict(X_train))))
print('Cat (test): ' + str(mean_absolute_error(y_test, model_cat_tun.predict(X_test))))



print('---')

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Lineal (test): ' + str(mean_absolute_error(y_test, linear.predict(X_test))))

print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Ridge (test): ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))

print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('LASSO (test): ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))

print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('ElasticNet (test): ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))

print('---')

print('SGD (train): ' + str(mean_absolute_error(y_train, reg.predict(X_train))))
print('SGD (test): ' + str(mean_absolute_error(y_test, reg.predict(X_test))))

In [ ]:
from sklearn.metrics import accuracy_score

print('Lineal (train): ' + str(linear.score(X_train, y_train)))
print('Lineal (test): ' + str(linear.score(X_test, y_test)))
print('---')
print('Ridge (train): ' + str(ridge.score(X_train, y_train)))
print('Ridge (test): ' + str(ridge.score(X_test, y_test)))
print('---')
print('LASSO (train): ' + str(lasso.score(X_train, y_train)))
print('LASSO (test): ' + str(lasso.score(X_test, y_test)))
print('---')
print('ElasticNet (train): ' + str(elasticnet.score(X_train, y_train)))
print('ElasticNet (test): ' + str(elasticnet.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print('Lineal (train): ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Lineal (test): ' + str(sqrt(mean_squared_error(y_test, linear.predict(X_test)))))
print('---')
print('Ridge (train): ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Ridge (test): ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))
print('---')
print('LASSO (train): ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('LASSO (test): ' + str(sqrt(mean_squared_error(y_test, lasso.predict(X_test)))))
print('---')
print('ElasticNet (train): ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('ElasticNet (test): ' + str(sqrt(mean_squared_error(y_test, elasticnet.predict(X_test)))))

Linear
---

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
fit_intercept = [True, False]
copy_x = [True, False]
normalize = [True, False]

In [ ]:
linear_Grid = GridSearchCV(
  estimator = LinearRegression(),
  param_grid = dict(fit_intercept = fit_intercept, copy_X = copy_x, normalize = normalize),
  cv = 5
)
linear_Grid.fit(X, y)

In [ ]:
linear_Grid.best_estimator_

In [ ]:
linear_Grid.best_params_

In [ ]:
linear_Grid.best_score_

In [ ]:
linear = LinearRegression(fit_intercept=True, normalize=False, copy_X=True)
linear.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, linear.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(linear.score(X_train, y_train)))
print("Test: " + str(linear.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, linear.predict(X_train)))))

Se puede observar que no mejora ya que los parámetros que nos da el GridSearch son los mismos que tiene de base.

Ridge
---

In [ ]:
ridge_Grid = GridSearchCV(
  estimator = Ridge(),
  param_grid = dict(alpha= np.linspace(0, 1, 50), copy_X = copy_x, fit_intercept=fit_intercept, normalize = normalize),
  cv = 5

)
ridge_Grid.fit(X, y)

In [ ]:
ridge_Grid.best_estimator_

In [ ]:
ridge_Grid.best_params_

In [ ]:
ridge_Grid.best_score_

In [ ]:
ridge = Ridge(alpha=0.32653061224489793, copy_X=True, fit_intercept=True,
      normalize=True, random_state=1337, solver='auto', tol=0.001)
ridge.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, ridge.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(ridge.score(X_train, y_train)))
print("Test: " + str(ridge.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, ridge.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_test, ridge.predict(X_test)))))

Parece que empeora ligeramente con respecto a los valores por defecto que tenía antes.

LASSO
---

In [ ]:
lasso_Grid = GridSearchCV(estimator = linear_model.Lasso(), param_grid = dict(alpha= np.linspace(0.05,1,20)), cv = 5)
lasso_Grid.fit(X, y)

In [ ]:
lasso_Grid.best_params_

In [ ]:
lasso = linear_model.Lasso(alpha=1, random_state=1337)
lasso.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, lasso.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, lasso.predict(X_train)))))

Se queda igual que con los valores de base.

ElasticNet
---

Este regresor a veces funciona y a veces explota, dejamos guardado los resultados de cuando ha funcionado. De todos modos nos sale el siguiente error:<br>


```
# /usr/local/lib/python3.6/dist-packages/sklearn linear_model_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge.
```



In [ ]:
elastic_Grid = GridSearchCV(
  estimator = ElasticNet(),
  param_grid = dict(alpha=np.linspace(0.1,1,2), l1_ratio=np.linspace(0,1,2)),
  cv = 5
)
elastic_Grid.fit(X, y)

In [ ]:
elastic_Grid.best_params_

In [ ]:
elasticnet = ElasticNet(alpha = 1.0, l1_ratio = 1, fit_intercept=True, normalize=False, copy_X=True)
elasticnet.fit(X_train, y_train)

In [ ]:
print("MAE: ")
print('Train: ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
print('Test: ' + str(mean_absolute_error(y_test, elasticnet.predict(X_test))))
print("---------------------------------------")
print("R^2: ")
print("Train: " + str(lasso.score(X_train, y_train)))
print("Test: " + str(lasso.score(X_test, y_test)))
print("---------------------------------------")
print("RMST: ")
print('Train: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))
print('Test: ' + str(sqrt(mean_squared_error(y_train, elasticnet.predict(X_train)))))

# Validación Cruzada
---

## Linear

MAE

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate

In [ ]:
cv_scores_linearMAE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearMAE.mean()}")

R^2

In [ ]:
cv_scores_linearR = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearR}")
print(f"Média métricas de validación cruzada: {cv_scores_linearR.mean()}")

RMSE

In [ ]:
cv_scores_linearRMSE = cross_val_score(
                estimator = linear,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_linearRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_linearRMSE.mean()}")

## Ridge
---

MAE

In [ ]:
cv_scores_ridgeMAE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeMAE.mean()}")

R^2

In [ ]:
cv_scores_ridgeR = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeR}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeR.mean()}")

RMSE

In [ ]:
cv_scores_ridgeRMSE = cross_val_score(
                estimator = ridge,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_ridgeRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_ridgeRMSE.mean()}")

## LASSO
---

MAE

In [ ]:
cv_scores_lassoMAE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoMAE.mean()}")

R^2

In [ ]:
cv_scores_lassoR = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoR}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoR.mean()}")

RMSE

In [ ]:
cv_scores_lassoRMSE = cross_val_score(
                estimator = lasso,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_lassoRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_lassoRMSE.mean()}")

## ElasticNet
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = elasticnet,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

## XGBoost
---

MAE

In [ ]:
cv_scores_netMAE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_absolute_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netMAE}")
print(f"Média métricas de validación cruzada: {cv_scores_netMAE.mean()}")

R^2

In [ ]:
cv_scores_netR = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'r2',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netR}")
print(f"Média métricas de validación cruzada: {cv_scores_netR.mean()}")

RMSE

In [ ]:
cv_scores_netRMSE = cross_val_score(
                estimator = xgb_model,
                X         = X,
                y         = y,
                scoring   = 'neg_mean_squared_error',
                cv        = 5
             )
print(f"Métricas validación cruzada: {cv_scores_netRMSE}")
print(f"Média métricas de validación cruzada: {cv_scores_netRMSE.mean()}")

# Prueba
---

In [ ]:
train = test
train['NombreCaballo'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

# Ajuste Preprocesamiento
- Borrar Puesto
- Ajuste Listas Apariciones

In [ ]:
train = train.drop('DistanciaMeta', axis=1)
train = train.drop('Dividendo', axis=1)

In [ ]:
train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]
print(listAparicionesHorse)
listTestHorse= eraseBlank(listaNombreCaballo)



train['Propietario'] = train['Propietario'].str.strip()
train['NombreCaballo'] = train['NombreCaballo'].str.strip()
train['Jinete'] = train['Jinete'].str.strip()
train['Preparador'] = train['Preparador'].str.strip()

train.loc[~train["Propietario"].isin(listAparicionesOwner), "Propietario"] = "OtherOwner"
# train.loc[~train["NombreCaballo"].isin(listAparicionesHorse), "NombreCaballo"] = "OtherHorse" #Lo hacemos mas tarde
train.loc[~train["Jinete"].isin(listAparicionesJockey), "Jinete"] = "OtherJockey"
train.loc[~train["Preparador"].isin(listAparicionesTrainer), "Preparador"] = "OtherTrainer"

# if 'OtherOwner' not in train:
#   train['OtherOwner'] = 0

# if 'OtherHorse' not in train:
#   train['OtherHorse'] = 0

# if 'OtherJockey' not in train:
#   train['OtherJockey'] = 0

# if 'OtherTrainer' not in train:
#   train['OtherTrainer'] = 0


# Preprocesamiento
---

In [ ]:
train['Jinete'] = train['Jinete'].str.partition('(')[0]
train['Preparador'] = train['Preparador'].str.partition('(')[0]
train['Propietario'] = train['Propietario'].str.partition('(')[0]
train['NombreCaballo'] = train['NombreCaballo'].str.partition('(')[0]

train.head(5)

## **Tratamiento (Peso)**

In [ ]:
# train['Peso'] = train['Peso'].str.partition('-')[0]
# train['Peso'] = train['Peso'].str[:2]

In [ ]:
train['Peso'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Fecha)**


In [ ]:
import datetime as dt
from datetime import date


def switchMonth(month):
  if month == 'ENERO':  
   return '01'
  elif month == 'FEBRERO':
    return '02'            
  elif month == 'MARZO':
   return '03'
  elif month == 'ABRIL':
    return '04' 
  elif month == 'MAYO':
    return '05'                  
  elif month == 'JUNIO':
    return '06'
  elif month == 'JULIO':
    return '07'
  elif month == 'AGOSTO':
    return '08'
  elif month == 'SEPTIEMBRE':
    return '09'
  elif month == 'OCTUBRE':
     return '10'
  elif month == 'NOVIEMBRE':
    return '11'
  elif month == 'DICIEMBRE':
    return '12'

def convertDate(fechas):
  index = 0
  for element in fechas:
    sum = 0
    year = element[-4:]
    dateAux = year
    day = element[:2]
    if(int(day) < 10):
      day = "0" + day[:1]
    month = element[5:]
    month = ''.join(month.split())[:-6].upper()
    month = switchMonth(month)
    dateAux = dateAux + "-" + month + "-" + day

    today = date.today()
    yearToday = int(str(today)[:4])
    monthToday = int(str(today)[5:7])
    dayToday = int(str(today)[8:10])
    if(yearToday > int(year)):
      sum += (yearToday - int(year))*365
    sum += (monthToday - int(month))*30
    sum += (dayToday - int(day))
    datetimeAux = dt.datetime(int(year), int(month), int(day))
    train.at[index, 'FechaAux'] = datetimeAux

    train.at[index, 'year'] = int(year)
    train.at[index, 'month'] = int(month)
    train.at[index, 'day'] = int(day)
    calculateSeason(int(month), index)

    train.at[index, 'DiasDesdeCarrera'] = sum
    index += 1
   
def calculateSeason(month, index):
  # train['Invierno'] = 0
  # train['Primavera'] = 0
  # train['Verano'] = 0
  train['Otoño'] = 0
  # if(month >= 12 and month <=2):
  #    train.at[index, 'Invierno'] = month
  # if(month >= 3 and month <=5):
  #    train.at[index, 'Primavera'] = month
  # if(month >= 6 and month <=8):
  #    train.at[index, 'Verano'] = month
  if(month >= 9 and month <=11):
     train.at[index, 'Otoño'] = month




t_array = train["Fecha"] 
convertDate(t_array)
train['FechaAux'].head(70)

In [ ]:
train.head(5)

In [ ]:
import datetime

def daysSincePreviousRace(nombres):
  index = 0
  for caballo in nombres:
    indexNombre = 0
    nombresAparece = []
    for otroCaballo in nombres:
      if(otroCaballo == caballo):
        nombresAparece.append(indexNombre)
      indexNombre += 1

    day_actual = train.at[index, 'day']
    month_actual = train.at[index, 'month']
    year_actual = train.at[index, 'year']

    fechaCaballoActual = dt.datetime(int(year_actual), int(month_actual), int(day_actual))
    fechaMasReciente = dt.datetime(2020, 1, 1)
    indexAux2 = 0
    fechaCambiada = False
    for aparicion in nombresAparece:
      if(train.at[aparicion, 'FechaAux'] < fechaCaballoActual): 
        if(indexAux2 == 0):
          fechaMasReciente = train.at[aparicion, 'FechaAux']
          fechaCambiada = True
          indexAux2 += 1
        elif(train.at[aparicion, 'FechaAux'] > fechaMasReciente):
          fechaMasReciente =  train.at[aparicion, 'FechaAux']
    if(fechaCambiada):
      diferenciaDias = (fechaCaballoActual - fechaMasReciente).days
    else:
      diferenciaDias = 45 # Penalización si no aparece en carrera anterior
    train.at[index, 'DaysSincePreviousRace'] = diferenciaDias
    index += 1


eventos = train['NombreCaballo']
daysSincePreviousRace(eventos)
train['DaysSincePreviousRace'].head(35)
# Recorro los nombres de caballos y guardo en un array los index de las columnas ligadas a un nombre de Caballo y despues recorrer las fechas y cuando coincida el array comprobar si ese dato
# es mas antiguo que la fecha de la fila actual y en el caso de serlo compruebo si es la más antigua de entre las anteriores participaciones











## Calculo Contricantes
---

In [ ]:
def calculoContrincantes(listaF, listaH, longitud):
  for fecha in listaF:
    for horario in listaH:
      index = 0
      listaIndex = []
      sum = 0
      while(index < longitud):
        if((train.at[index, 'Fecha'] == fecha ) and (train.at[index, 'Hora'] == horario )):
          listaIndex.append(index)
          sum += 1
        index += 1
      for indice in listaIndex:
        train.at[indice, 'Contrincantes'] = sum


listaFechas = train['Fecha'].unique().tolist()
listaHorarios = train['Hora'].unique().tolist()
longitud = len(train.index)

calculoContrincantes(listaFechas, listaHorarios, longitud)




## **Busqueda Caballos Ganadores en ciertas condiciones**
---

### Busqueda UltimasActuacionesAlter
---

In [ ]:
def ultimasActuacionesIter(indices, indicesOg):                      
  for i in range(indices):
    print(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0.0
    sum = 0.0

    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if((nombreAux == nombre and (fechaAux < fecha))):
        print("Valor Puesto-->", trainOg.at[j, 'Puesto'], "/", fechaAux)
        media += (trainOg.at[j, 'Puesto'] )
        sum += 1
    print("Suma Medias -->", media, "/", sum, "<-- Nº Act Anteriores")
    if(sum != 0):
      media /= sum
    if(sum == 0):
      media = 2.5
      train.at[i, 'UltimasActuacionesAlter'] = media
    else:
      train.at[i, 'UltimasActuacionesAlter'] = media
    print(i, "-->", nombre, " MediaUltAct->", media) 

        
indicesOg = len(trainOg.index)
indices = len(train.index)
ultimasActuacionesIter(indices, indicesOg)

In [ ]:
train['UltimasActuacionesAlter'][:20].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Distancia
---

In [ ]:
pd.set_option('display.max_rows', None)
train.head(70)

In [ ]:
def comprobacionTipoDistanciaTrain(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = trainOg.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;

def comprobacionTipoDistanciaTest(indice):
  tipo = 0   # 0, 1, 2 / Corta, Media, Larga
  val = train.at[indice, 'Distancia']
  if(val >= 3200):
    tipo = 2
  if((val >= 1600) & (val <= 2400)):
    tipo = 1
  return tipo;
  
def destrezaDistanciaTest(indices,indicesOg):                      
  for i in range(indices):
    tipo = comprobacionTipoDistanciaTest(i)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoDistanciaTrain(j)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'])
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'])
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones en esas condiciones y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaDistancia'] = 1
    else:
      train.at[i, 'DestrezaDistancia'] = 0
        
    #IMP! Valorar si es buena idea crear una columna a partir de la variable "media" + "mediaAux" obtenida aqui.

indices = len(train.index)
indicesOg = len(trainOg.index)
destrezaDistanciaTest(indices, indicesOg)



#Len Intervalos
print(len(train[(train['Distancia'] < 1600).tolist()]))
print(len(train[((train['Distancia'] >= 1600) & (train['Distancia'] <= 2400) ).tolist()]))
print(len(train[(train['Distancia'] >= 3200).tolist()]))

In [ ]:
pd.set_option('display.max_rows', None)
train[['NombreCaballo','DestrezaDistancia']].head(64)

### Busqueda Meteorologia
---

In [ ]:
train['Meteorología'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoMeteorologia(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado  
  val = train.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "Llovizna" in val or "precipitaciones" in val or "lluvias" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def comprobacionTipoMeteorologiaOg(indice, listaMeteo):
  tipo = -1 # 0 -> Lluvia / 1 -> Nublado / 2 -> Soleado, Despejado  
  val = trainOg.at[indice, 'Meteorología']
  for meteo in listaMeteo:
   if("Lluvia" in val or "Llovizna" in val or "precipitaciones" in val or "lluvias" in val):
      tipo = 1
   elif("Soleado" in val or "Despejado" in val):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaMeteorologia(indices, indicesOg, listaMeteo):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoMeteorologia(i, listaMeteo)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoMeteorologiaOg(j, listaMeteo)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaMeteo'] = 1
      print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    else:
      train.at[i, 'DestrezaMeteo'] = 0
        
indicesOg = len(trainOg.index)
indices = len(train.index)
listaMeteo = train['Meteorología'].unique().tolist()
destrezaMeteorologia(indices, indicesOg,  listaMeteo)

### Busqueda Lluvia
---

In [ ]:
train['LLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoLLuvia(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1 
  val = train.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;


def comprobacionTipoLLuviaOg(indice, listaLLuvia):
  tipo = -1 # 0 -> 0.0 / 1 -> > 0.0 / 2 -> > 1 
  val = trainOg.at[indice, 'LLuvia']
  for meteo in listaMeteo:
   if(val > 0):
      tipo = 1
   elif(val > 1):
      tipo = 2
   else:
     tipo = 0
  return tipo;

def destrezaLLuvia(indices, indicesOg,  listaLLuvia):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoLLuvia(i, listaLLuvia)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoLLuviaOg(j, listaLLuvia)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaLLuvia'] = 1
    else:
      train.at[i, 'DestrezaLLuvia'] = 0
        
indicesOg= len(trainOg.index)
indices = len(train.index)
listaLLuvia = train['LLuvia'].unique().tolist()
destrezaLLuvia(indices, indicesOg,  listaLLuvia)

In [ ]:
train['DestrezaLLuvia'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Temperatura
---

In [ ]:
train['Temperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTemperatura(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = train.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def comprobacionTipoTemperaturaOg(indice, listaTemperatura):
  tipo = -1 # 0 -> < 20 / 1 -> > 30 > x < 20 / 2 -> > > 30
  val = trainOg.at[indice, 'Temperatura']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 20):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaTemperatura(indices, indicesOg, listaTemperatura, ):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTemperatura(i, listaTemperatura)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTemperaturaOg(j, listaTemperatura)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTemperatura'] = 1
    else:
      train.at[i, 'DestrezaTemperatura'] = 0
        
indicesOg = len(trainOg.index)
indices = len(train.index)
listaTemperatura = train['Temperatura'].unique().tolist()
destrezaTemperatura(indices, indicesOg,  listaTemperatura)

In [ ]:
train['DestrezaTemperatura'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Viento
---

In [ ]:
train['Viento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoViento(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = train.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;


def comprobacionTipoVientoOg(indice, listaViento):
  tipo = -1 # 0 -> < 5 / 1 -> > 30 > x > 5 / 2 -> > > 30
  val = trainOg.at[indice, 'Viento']
  for meteo in listaMeteo:
   if(val > 30):
      tipo = 2
   elif(val < 5):
      tipo = 0
   else:
     tipo = 0
  return tipo;

def destrezaViento(indices, indicesOg, listaViento):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoViento(i, listaViento)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoVientoOg(j, listaViento)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaViento'] = 1
    else:
      train.at[i, 'DestrezaViento'] = 0
        
indicesOg =  len(trainOg.index)
indices = len(train.index)
listaViento = train['Viento'].unique().tolist()
destrezaViento(indices,indicesOg, listaViento)

In [ ]:
train['DestrezaViento'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### Busqueda Terreno
---

In [ ]:
train['Terreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
def comprobacionTipoTerreno(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = train.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    element = element.replace('H-Bueno', 'H - Bueno')
    train.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'H - Blando'):
      tipo = 0
   else:
      tipo = 4
  return tipo;

def comprobacionTipoTerrenoOg(indice, listaTerreno):
  tipo = -1 # 0 -> Arena / 1 -> H - Blando / 2 -> H - Bueno
  val = trainOg.at[indice, 'Terreno']
  if(val == "H-Bueno"):
    element = element.replace('H-Bueno', 'H - Bueno')
    trainOg.at[indice, 'Terreno'] = val
  for meteo in listaMeteo:
   if(val == 'H - Bueno'):
      tipo = 2
   elif(val == 'H - Blando'):
      tipo = 1
   elif(val == 'H - Blando'):
      tipo = 0
   else:
      tipo = 4
  return tipo;

def destrezaTerreno(indices,indicesOg, listaTerreno):                      
  for i in range(indices):
    print(i)
    tipo = comprobacionTipoTerreno(i, listaTerreno)
    nombre = train.at[i, 'NombreCaballo']
    fecha = train.at[i, 'FechaAux']
    media = 0 # Media carreras
    sum = 0 # Numero Carreras
    mediaAux = 0 
    sumAux = 0
    for j in range(indicesOg):
      fechaAux = trainOg.at[j, 'FechaAux']
      tipoAux = comprobacionTipoTerrenoOg(j, listaTerreno)
      nombreAux= trainOg.at[j, 'NombreCaballo']
      if(nombreAux == nombre):
        if((fechaAux < fecha) & (tipoAux == tipo)):
          media += (trainOg.at[j, 'Puesto'] )
          sum += 1
        elif((fechaAux < fecha) & (tipoAux != tipo)):
          mediaAux += (trainOg.at[j, 'Puesto'] )
          sumAux += 1
    if(sum != 0):
      media /= sum
    if(sumAux != 0):
      mediaAux /= sumAux
    print(i, "-->", nombre, " - Media->", media, "/MediaAux-->", mediaAux, "/Numero Destreza(", sum, sumAux, "):Numero Otros/") 
    if((sum >= 2) & (sumAux >= 2) & (((1.1* media) < mediaAux))): # Si hay mas de 2 participaciones (en esas condiciones y resto ) y con una media al menos un 33% mas baja , es valido
      train.at[i, 'DestrezaTerreno'] = 1
    else:
      train.at[i, 'DestrezaTerreno'] = 0
        

indices = len(train.index)
indicesOg = len(trainOg.index)
listaTerreno = train['Terreno'].unique().tolist()
destrezaTerreno(indices,indicesOg,  listaTerreno)

In [ ]:
train['DestrezaTerreno'].value_counts().sort_values(ascending=False) #90 primeras apariencias >= 3

### **Tratamiento (CaballoDistintos)**

In [ ]:
train.loc[~train["NombreCaballo"].isin(listAparicionesHorse), "NombreCaballo"] = "OtherHorse" # Lo posponemos a este momento(por ahora)

## **Tratamiento (Edad)**

In [ ]:
train['Edad'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Mantilla)**

In [ ]:
train['Mantilla'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

## **Tratamiento (Problemas)**


In [ ]:
train['Problemas'].value_counts()[:35].sort_values(ascending=False) #90 primeras apariencias >= 3

In [ ]:
train['Problemas'] = train['Problemas'].fillna('0')
train['Problema_Nulo'] = np.where(train['Problemas'].str.contains('0'),1,0)
train['Problema_1'] = np.where(train['Problemas'].str.contains('1'),1,0)
train['Problema_2'] = np.where(train['Problemas'].str.contains('2'),1,0)
train['Problema_3'] = np.where(train['Problemas'].str.contains('3'),1,0)
train['Problema_4'] = np.where(train['Problemas'].str.contains('4'),1,0)
train['Problema_5'] = np.where(train['Problemas'].str.contains('5'),1,0)
train['Problema_6'] = np.where(train['Problemas'].str.contains('6'),1,0)
train['Problema_7'] = np.where(train['Problemas'].str.contains('7'),1,0)
train['Problema_8'] = np.where(train['Problemas'].str.contains('8'),1,0)
train.head()



## **Tratamiento (UltimasActuaciones)**


In [ ]:
# def count_lastRaces(arrayLast):
#   index = 0
#   for element in arrayLast:
#     element = element.replace(',', '')
#     element = element.replace('[', '')
#     element = element[:len(element)-1]
#     sum = 0
#     if 'Debutante' in element:
#       sum = 10
#       element = []
#     else:
#       element = element.replace('NP', '08')
#       element = element.replace('Desc', '08')
#       element = element.replace(' ', '')
#       i = len(element)
#       j = len(element)
#       iter = (len(element)) / 2
#       count = 0
#       while(iter > 0):
#         if(count == 0):
#           i = i-2
#           sum += int(element[i:j])**1.3
#           j = j-2
#         elif(count == 1):
#           i = i-2
#           sum += int(element[i:j])**1.2
#           j = j-2
#         else:
#           print(iter)
#           i = i-2
#           sum += int(element[i:j])
#           j = j-2
#         count += 1
#         iter -= 1
#       sum = sum / len(element)
#       if(index < 30):
#         print('******INDEX ', index+2, '*****')
#         print('Sum -->', sum, '/Len -->', len(element))
#         print('Result -->', sum, '/Element -->' ,element)
#     train.at[index, 'MediaUltimasActuaciones'] = sum
#     train.at[index, 'CantidadActuaciones'] = len(element)
#     index += 1

      



# t_array = train["UltimasActuaciones"]
# count_lastRaces(t_array)

In [ ]:
#  train['MediaUltimasActuaciones'].head(25)

## **Tratamiento (Hora)**


In [ ]:
train['Hora'] = train['Hora'].str.replace(':', '') 

## **Tratamiento (Terreno)**


## **Tratamiento (Distancia)**


## **Tratamiento (Tipo)**


## **Tratamiento (SentidoHipodromo)**


## **Tratamiento (Meteorologia)**


## **Tratamiento (Lluvia)**


## **Tratamiento (Viento)**


## **Tratamiento (TemperaturaMax)**


## **Tratamiento (TemperaturaMin)**


## **Tratamiento (Hipodromo)**


#Continuacion

In [ ]:
X_val = train
ids = train
X_val = X_val.reset_index()
X_val.info()
X_valTF = X_val #TensorFlow

In [ ]:
column_transformer_test = sklearn.compose.ColumnTransformer(transformers=[
    ("drop", "drop", ['Fecha','year','day', 'Problemas','Hora', 'FechaAux', 'Mantilla', 'Terreno', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'Temperatura', 'Hipodromo', 'Contrincantes']),
    ("scale", sklearn.preprocessing.StandardScaler(), []),
    # ("num", sklearn.preprocessing.MinMaxScaler(), ['Distancia', 'Edad', 'Peso']),
    ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Propietario', 'Preparador', 'Jinete']),
], remainder='passthrough');

# column_transformer = sklearn.compose.ColumnTransformer(transformers=[
#     ("drop", "drop", ['UltimasActuaciones','Fecha','year','day', 'Problemas']),
#     ("scale", sklearn.preprocessing.StandardScaler(), []),
#     ("one-hot", sklearn.preprocessing.OneHotEncoder(handle_unknown="ignore"), ['NombreCaballo','Peso','Edad', 'Mantilla','Propietario', 'Preparador', 'Jinete', 'estacion', 'FechaAux', 'Terreno','Distancia', 'Tipo', 'Meteorología', 'LLuvia', 'Viento', 'TemperaturaMax', 'TemperaturaMin', 'Hipodromo']),
# ], remainder='passthrough');

In [ ]:
train = train.reset_index()

X_val = column_transformer_test.fit_transform(X_val)

In [ ]:
X_val

In [ ]:
column_transformer_train.get_feature_names()

In [ ]:
column_transformer_test.get_feature_names()

# Algoritmos
---

In [ ]:
from sklearn.metrics import mean_absolute_error
tree.fit(X, y);

print('---')

print('Tree (train): ' + str(mean_absolute_error(y_train, tree.predict(X_train))))
predicted_yTree = tree.predict(X_val)
print('Tree (test): ' + str(mean_absolute_error(test_labels, tree.predict(X_val))))

print('---')

print('RF (train): ' + str(mean_absolute_error(y_train, rf.predict(X_train))))
predicted_yRF = rf.predict(X_val)
print('RF (test): ' + str(mean_absolute_error(test_labels, rf.predict(X_val))))

print('---')

print('XGB (train): ' + str(mean_absolute_error(y_train, xgb_model.predict(X_train))))
predicted_yXGB = xgb_model.predict(X_val)
print('XGB (test): ' + str(mean_absolute_error(test_labels, xgb_model.predict(X_val))))


print('---')

print('Cat (train): ' + str(mean_absolute_error(y_train, model_cat_tun.predict(X_train))))
predicted_yCat = model_cat_tun.predict(X_val)
print('Cat (test): ' + str(mean_absolute_error(test_labels, model_cat_tun.predict(X_val))))

print('---')

print('Lineal (train): ' + str(mean_absolute_error(y_train, linear.predict(X_train))))
predicted_yLineal = linear.predict(X_val)
print('Lineal (test): ' + str(mean_absolute_error(test_labels, linear.predict(X_val))))



print('---')

print('Ridge (train): ' + str(mean_absolute_error(y_train, ridge.predict(X_train))))
predicted_yRidge = ridge.predict(X_val)
print('Ridge (test): ' + str(mean_absolute_error(test_labels, ridge.predict(X_val))))


print('---')

print('LASSO (train): ' + str(mean_absolute_error(y_train, lasso.predict(X_train))))
predicted_yLASSO = lasso.predict(X_val)
print('LASSO (test): ' + str(mean_absolute_error(test_labels, lasso.predict(X_val))))


print('---')

print('ElasticNet (train): ' + str(mean_absolute_error(y_train, elasticnet.predict(X_train))))
predicted_yElastic = elasticnet.predict(X_val)
print('ElasticNet (test): ' + str(mean_absolute_error(test_labels, elasticnet.predict(X_val))))

In [ ]:
import graphviz 
from sklearn.tree import export_graphviz

dot_data = export_graphviz(tree, out_file=None) 
graph = graphviz.Source(dot_data)

In [ ]:
trainOg = train
train['Prediccion'] = 0
train['Puesto'] = 0
train['Carrera'] = 0
y_val = pd.concat([train['Carrera'], train['Puesto'],train['NombreCaballo'], train['Prediccion'], train['UltimasActuacionesAlter'],train['DestrezaDistancia'], train['FechaAux'], train['Hora']],axis=1)

In [ ]:
def ordenarTest(horarios, index, x):
  carrera = 0
  aux = -1
  for horario in horarios:
    if(aux != horario):
      carrera += 1
      aux = horario
    dict = {}
    conta = 0
    for i in index:
      horarioActual = x.at[i, 'Hora']
      if(horario == horarioActual ):
          x.at[i, 'Carrera'] = carrera
          prediccion = x.at[i, 'Prediccion']
          dict[i] = prediccion
          conta += 1
    puesto = 1
    for w in sorted(dict, key=dict.get):
      x.at[w, ['Puesto']] = puesto
      puesto += 1

def ordenarTestReverse(horarios, index, x):
  carrera = 0
  aux = -1
  for horario in horarios:
    if(aux != horario):
      carrera += 1
      aux = horario
    dict = {}
    conta = 0
    for i in index:
      horarioActual = x.at[i, 'Hora']
      if(horario == horarioActual ):
          x.at[i, 'Carrera'] = carrera
          prediccion = x.at[i, 'Prediccion']
          dict[i] = prediccion
          conta += 1
    puesto = 1
    for w in sorted(dict, key=dict.get, reverse=True):
      x.at[w, ['Puesto']] = puesto
      puesto += 1    
    


horarios = train['Hora'].tolist()
index = train['index'].tolist()

In [ ]:
y_valRF = y_val
y_valXgb = y_val
y_valCat = y_val
y_val1 = y_val
y_val2= y_val
y_val3 = y_val
y_val4= y_val

In [ ]:
y_valRF['Prediccion'] = predicted_yRF
xRF = y_valRF
ordenarTest(horarios, index, xRF)
xRF.to_csv('outputRF.csv' , index=False)

In [ ]:
y_valXgb['Prediccion'] = predicted_yXGB
xXGB = y_valXgb
ordenarTest(horarios, index, xXGB)
xXGB.to_csv('outputXgb.csv' , index=False)


In [ ]:
y_valCat['Prediccion'] = predicted_yCat
xCat = y_valCat
ordenarTest(horarios, index, xCat)
xCat.to_csv('outputCat.csv' , index=False)

In [ ]:

y_val1['Prediccion'] = predicted_yLineal
x1 = y_val1
ordenarTest(horarios, index, x1)
x1.to_csv('outputLineal.csv' , index=False)
y_val2['Prediccion'] = predicted_yRidge
x2 = y_val2
ordenarTestReverse(horarios, index, x2)
x2.to_csv('outputRidge.csv', index=False)
y_val3['Prediccion'] = predicted_yLASSO
x3 = y_val3
ordenarTest(horarios, index, x3)
x3.to_csv('outputLASSO.csv', index=False)
y_val4['Prediccion'] = predicted_yElastic
x4 = y_val4
ordenarTest(horarios, index, x4)
x4.to_csv('outputElastic.csv' , index=False)

In [ ]:
pd.set_option('display.max_columns', None)
x4.head(70)

- **Considero importante realizar una explicacion en cada subida de .csv sobre en que se diferencia respecto al anterior**